In [ ]:
%pip install torch

In [ ]:
import numpy as np

image_dataset = '../datasets/unlabelled_train_data_images.npy'


In [ ]:
import numpy as np
import torch
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import os

class ImagePreprocessor:
    """
    A class to preprocess and label an unlabeled MNIST-like dataset using PyTorch and scikit-learn,
    with t-SNE visualization on a subset of the data.
    """
    
    def __init__(self, data_path, n_clusters=10, random_state=42):
        """
        Initialize the preprocessor.
        
        Args:
            data_path (str): Path to the .npy file containing the unlabeled images.
            n_clusters (int): Number of clusters for K-Means (default: 10 for digits 0-9).
            random_state (int): Random seed for reproducibility.
        """
        self.data_path = data_path
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.images = None
        self.preprocessed_images = None
        self.labels = None
        
    def load_data(self):
        """
        Load the dataset from the .npy file.
        
        Returns:
            numpy.ndarray: Loaded images with shape (n_samples, 1, 28, 28).
        """
        self.images = np.load(self.data_path)
        print(f"Loaded data shape: {self.images.shape}")
        
        # Validate data
        if self.images.shape[1:] != (1, 28, 28):
            raise ValueError("Expected images with shape (n_samples, 1, 28, 28)")
        if np.any(np.isnan(self.images)) or np.any(np.isinf(self.images)):
            raise ValueError("Data contains NaN or infinite values")
        
        return self.images
    
    def preprocess_images(self):
        """
        Preprocess the images: normalize and convert to PyTorch tensors.
        
        Returns:
            torch.Tensor: Preprocessed images with shape (n_samples, 1, 28, 28).
        """
        if self.images is None:
            self.load_data()
        
        # Normalize to [ureate the labels
        self.preprocessed_images = self.images.astype(np.float32) / 255.0
        print(f"Preprocessed data shape: {self.preprocessed_images.shape}, range: [{self.preprocessed_images.min()}, {self.preprocessed_images.max()}]")
        
        # Convert to PyTorch tensor
        tensor_images = torch.from_numpy(self.preprocessed_images)
        return tensor_images
    
    def flatten_for_clustering(self, apply_pca=False, n_components=50):
        """
        Flatten images for clustering and optionally apply PCA.
        
        Args:
            apply_pca (bool): Whether to apply PCA for dimensionality reduction.
            n_components (int): Number of PCA components if apply_pca is True.
        
        Returns:
            numpy.ndarray: Flattened or PCA-transformed images with shape (n_samples, n_features).
        """
        if self.preprocessed_images is None:
            self.preprocess_images()
        
        # Flatten images: (n_samples, 1, 28, 28) -> (n_samples, 784)
        flattened = self.preprocessed_images.reshape(self.preprocessed_images.shape[0], -1)
        print(f"Flattened data shape: {flattened.shape}")
        
        if apply_pca:
            pca = PCA(n_components=n_components, random_state=self.random_state)
            flattened = pca.fit_transform(flattened)
            print(f"PCA-transformed data shape: {flattened.shape}, explained variance ratio: {np.sum(pca.explained_variance_ratio_):.4f}")
        
        return flattened
    
    def label_with_kmeans(self, apply_pca=False, n_components=50):
        """
        Label the dataset using K-Means clustering.
        
        Args:
            apply_pca (bool): Whether to apply PCA before clustering.
            n_components (int): Number of PCA components if apply_pca is True.
        
        Returns:
            numpy.ndarray: Cluster labels with shape (n_samples,).
        """
        flattened_images = self.flatten_for_clustering(apply_pca, n_components)
        
        # Apply K-Means
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state, n_init=10)
        self.labels = kmeans.fit_predict(flattened_images)
        print(f"Generated {self.n_clusters} clusters")
        
        # Evaluate clustering quality
        if self.images.shape[0] < 10000:  # Silhouette score is computationally expensive
            score = silhouette_score(flattened_images, self.labels)
            print(f"Silhouette score: {score:.4f}")
        
        return self.labels
    
    def visualize_clusters(self, save_path=None):
        """
        Visualize a few images from each cluster.
        
        Args:
            save_path (str, optional): Path to save the visualization plot.
        """
        if self.labels is None:
            raise ValueError("Run label_with_kmeans() first to generate labels")
        
        fig, axes = plt.subplots(self.n_clusters, 5, figsize=(15, 2 * self.n_clusters))
        for cluster in range(self.n_clusters):
            # Get indices of images in this cluster
            cluster_indices = np.where(self.labels == cluster)[0]
            # Select up to 5 random images
            selected_indices = np.random.choice(cluster_indices, size=min(5, len(cluster_indices)), replace=False)
            for i, idx in enumerate(selected_indices):
                axes[cluster, i].imshow(self.preprocessed_images[idx, 0], cmap='gray')
                axes[cluster, i].axis('off')
                if i == 0:
                    axes[cluster, i].set_title(f"Cluster {cluster}")
        
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            print(f"Cluster visualization saved to {save_path}")
        plt.show()
    
    def visualize_tsne(self, subset_ratio=0.2, apply_pca=True, n_components=50, save_path=None):
        """
        Apply t-SNE to visualize 20% of the dataset in 2D, colored by K-Means labels.
        
        Args:
            subset_ratio (float): Fraction of the dataset to use (default: 0.2 for 20%).
            apply_pca (bool): Whether to apply PCA before t-SNE.
            n_components (int): Number of PCA components if apply_pca is True.
            save_path (str, optional): Path to save the t-SNE visualization plot.
        
        Returns:
            numpy.ndarray: 2D t-SNE embeddings.
        """
        if self.labels is None:
            raise ValueError("Run label_with_kmeans() first to generate labels")
        
        # Select 20% of the data randomly
        n_samples = int(self.preprocessed_images.shape[0] * subset_ratio)
        indices = np.random.choice(self.preprocessed_images.shape[0], size=n_samples, replace=False)
        subset_images = self.preprocessed_images[indices]
        subset_labels = self.labels[indices]
        print(f"Selected {n_samples} samples for t-SNE visualization")
        
        # Flatten or apply PCA
        flattened = subset_images.reshape(n_samples, -1)
        if apply_pca:
            pca = PCA(n_components=n_components, random_state=self.random_state)
            flattened = pca.fit_transform(flattened)
            print(f"PCA-transformed subset shape: {flattened.shape}, explained variance ratio: {np.sum(pca.explained_variance_ratio_):.4f}")
        
        # Apply t-SNE
        tsne = TSNE(n_components=2, random_state=self.random_state, n_jobs=-1)
        tsne_embeddings = tsne.fit_transform(flattened)
        print(f"t-SNE embeddings shape: {tsne_embeddings.shape}")
        
        # Visualize
        plt.figure(figsize=(10, 8))
        scatter = plt.scatter(tsne_embeddings[:, 0], tsne_embeddings[:, 1], c=subset_labels, cmap='tab10', alpha=0.6)
        plt.colorbar(scatter, label='Cluster Label')
        plt.title('t-SNE Visualization of 20% MNIST Dataset')
        plt.xlabel('t-SNE Component 1')
        plt.ylabel('t-SNE Component 2')
        
        if save_path:
            plt.savefig(save_path)
            print(f"t-SNE visualization saved to {save_path}")
        plt.show()
        
        return tsne_embeddings
    
    def save_labeled_dataset(self, output_path):
        """
        Save the preprocessed images and labels to a .npz file.
        
        Args:
            output_path (str): Path to save the .npz file.
        """
        if self.preprocessed_images is None or self.labels is None:
            raise ValueError("Preprocess images and generate labels first")
        
        np.savez(output_path, images=self.preprocessed_images, labels=self.labels)
        print(f"Labeled dataset saved to {output_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

class ImageLabelModifier:
    """
    A class to manually modify labels in a labeled MNIST-like dataset saved as a .npz file,
    with support for iterative cluster-to-digit mapping updates.
    """
    
    def __init__(self, npz_path, random_state=42):
        """
        Initialize the label modifier.
        
        Args:
            npz_path (str): Path to the .npz file containing images and labels.
            random_state (int): Random seed for reproducibility in visualizations.
        """
        self.npz_path = npz_path
        self.random_state = random_state
        self.images = None
        self.labels = None
        self.n_clusters = 10  # Assuming 10 digit classes
        
    def load_npz_data(self):
        """
        Load images and labels from the .npz file.
        
        Returns:
            tuple: (images, labels) as numpy arrays.
        """
        with np.load(self.npz_path) as data:
            self.images = data['images']
            self.labels = data['labels']
        
        print(f"Loaded data: images shape {self.images.shape}, labels shape {self.labels.shape}")
        
        # Validate data
        if self.images.shape[1:] != (1, 28, 28):
            raise ValueError("Expected images with shape (n_samples, 1, 28, 28)")
        if self.labels.shape != (self.images.shape[0],):
            raise ValueError("Labels shape does not match images")
        if not np.all(np.isin(self.labels, np.arange(self.n_clusters))):
            raise ValueError("Labels contain values outside expected range [0, 9]")
        
        return self.images, self.labels
    
    def visualize_current_labels(self, samples_per_cluster=5, save_path=None):
        """
        Visualize a few images from each cluster to inspect current labels.
        
        Args:
            samples_per_cluster (int): Number of images to show per cluster.
            save_path (str, optional): Path to save the visualization plot.
        """
        if self.labels is None:
            self.load_npz_data()
        
        np.random.seed(self.random_state)
        fig, axes = plt.subplots(self.n_clusters, samples_per_cluster, figsize=(3 * samples_per_cluster, 2 * self.n_clusters))
        
        for cluster in range(self.n_clusters):
            # Get indices of images in this cluster
            cluster_indices = np.where(self.labels == cluster)[0]
            # Select up to samples_per_cluster random images
            selected_indices = np.random.choice(cluster_indices, size=min(samples_per_cluster, len(cluster_indices)), replace=False)
            for i, idx in enumerate(selected_indices):
                axes[cluster, i].imshow(self.images[idx, 0], cmap='gray')
                axes[cluster, i].axis('off')
                if i == 0:
                    axes[cluster, i].set_title(f"Cluster {cluster}")
        
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
            print(f"Current labels visualization saved to {save_path}")
        plt.show()
    
    def map_cluster_to_digit(self, cluster_to_digit_map):
        """
        Apply a manual mapping from cluster IDs to true digit labels.
        
        Args:
            cluster_to_digit_map (dict): Mapping from cluster ID to digit (e.g., {0: 5, 1: 2, ...}).
        
        Returns:
            numpy.ndarray: Updated labels with shape (n_samples,).
        """
        if self.labels is None:
            self.load_npz_data()
        
        # Validate mapping
        if not all(k in range(self.n_clusters) for k in cluster_to_digit_map.keys()):
            raise ValueError("Cluster IDs in mapping must be in range [0, 9]")
        if not all(v in range(10) for v in cluster_to_digit_map.values()):
            raise ValueError("Digit values in mapping must be in range [0, 9]")
        if len(set(cluster_to_digit_map.values())) != len(cluster_to_digit_map):
            raise ValueError("Mapping must assign unique digits to clusters")
        
        # Create new labels array
        new_labels = np.copy(self.labels)
        for cluster, digit in cluster_to_digit_map.items():
            new_labels[self.labels == cluster] = digit
        
        self.labels = new_labels
        print("Applied cluster-to-digit mapping")
        return self.labels
    
    def get_user_mapping(self):
        """
        Prompt the user to input a cluster-to-digit mapping.
        
        Returns:
            dict: Mapping from cluster ID to digit, or None if user skips.
        """
        print("\nEnter cluster-to-digit mapping (e.g., '0:5, 1:2, ...') or press Enter to skip.")
        print("Format: 'cluster:digit' pairs separated by commas, covering all clusters 0-9.")
        user_input = input("Mapping: ").strip()
        
        if not user_input:
            print("No mapping provided, keeping current labels")
            return None
        
        try:
            # Parse input (e.g., "0:5, 1:2, 2:1, ...")
            mapping = {}
            pairs = user_input.split(',')
            for pair in pairs:
                cluster, digit = map(int, pair.strip().split(':'))
                if cluster not in range(self.n_clusters) or digit not in range(10):
                    raise ValueError("Invalid cluster or digit value")
                mapping[cluster] = digit
            
            # Validate completeness and uniqueness
            if len(mapping) != self.n_clusters:
                raise ValueError(f"Mapping must include all {self.n_clusters} clusters")
            if len(set(mapping.values())) != self.n_clusters:
                raise ValueError("Each cluster must map to a unique digit")
            
            return mapping
        except Exception as e:
            print(f"Error parsing mapping: {e}. Please try again.")
            return None
    
    def iterative_update_labels(self, samples_per_cluster=5, vis_path_prefix='labels_visualization'):
        """
        Iteratively visualize clusters, prompt for mapping, and update labels.
        
        Args:
            samples_per_cluster (int): Number of images to show per cluster.
            vis_path_prefix (str): Prefix for visualization save paths.
        
        Returns:
            numpy.ndarray: Final updated labels.
        """
        if self.labels is None:
            self.load_npz_data()
        
        iteration = 0
        while True:
            # Visualize current labels
            vis_path = f"../datasets/{vis_path_prefix}_iter{iteration}.png"
            self.visualize_current_labels(samples_per_cluster=samples_per_cluster, save_path=vis_path)
            
            # Get user mapping
            mapping = self.get_user_mapping()
            if mapping:
                # Apply mapping
                self.map_cluster_to_digit(mapping)
            else:
                print("No changes made to labels")
            
            # Ask if user wants to continue
            response = input("\nDo you want to refine the mapping further? (yes/no): ").strip().lower()
            if response != 'yes':
                break
            
            iteration += 1
        
        # Final visualization
        final_vis_path = f"../datasets/{vis_path_prefix}_final.png"
        self.visualize_current_labels(samples_per_cluster=samples_per_cluster, save_path=final_vis_path)
        print("Final labels visualization completed")
        return self.labels
    
    def save_updated_dataset(self, output_path):
        """
        Save the images and updated labels to a new .npz file.
        
        Args:
            output_path (str): Path to save the updated .npz file.
        """
        if self.images is None or self.labels is None:
            raise ValueError("Load data and modify labels first")
        
        np.savez(output_path, images=self.images, labels=self.labels)
        print(f"Updated dataset saved to {output_path}")





In [ ]:

def processImages():
    data_path = '../datasets/unlabelled_train_data_images.npy'
    output_path = '../datasets/labeled_train_data.npz'
    cluster_vis_path = '../datasets/cluster_visualization.png'
    tsne_vis_path = '../datasets/tsne_visualization.png'
    
    preprocessor = ImagePreprocessor(data_path, n_clusters=10, random_state=42)
    
    # Load and preprocess
    preprocessor.load_data()
    tensor_images = preprocessor.preprocess_images()
    
    # Label using K-Means with PCA
    labels = preprocessor.label_with_kmeans(apply_pca=True, n_components=50)
    
    # Visualize clusters
    preprocessor.visualize_clusters(save_path=cluster_vis_path)
    
    # Visualize t-SNE on 20% of the data
    tsne_embeddings = preprocessor.visualize_tsne(subset_ratio=0.2, apply_pca=True, n_components=50, save_path=tsne_vis_path)
    
    # Save the labeled dataset
    preprocessor.save_labeled_dataset(output_path)
    
    # Verify saved data
    with np.load(output_path) as data:
        saved_images = data['images']
        saved_labels = data['labels']
        print(f"Saved data: images shape {saved_images.shape}, labels shape {saved_labels.shape}")


processImages()

In [ ]:

# def postProcessImages():
#     npz_path = '../datasets/labeled_train_data.npz'
#     output_path = '../datasets/updated_labeled_train_data.npz'
#     vis_path = '../datasets/current_labels_visualization.png'
    
#     modifier = ImageLabelModifier(npz_path, random_state=42)
    
#     # Load the .npz file
#     images, labels = modifier.load_npz_data()
    
#     # Visualize current labels to inspect clusters
#     modifier.visualize_current_labels(samples_per_cluster=5, save_path=vis_path)
    
#     # Example: Define a cluster-to-digit mapping based on visualization
#     # This is a placeholder; replace with actual mapping after inspecting visualization
#     cluster_to_digit_map = {
#         0: 1,  # Cluster 0 corresponds to digit 5
#         1: 6,  # Cluster 1 corresponds to digit 2
#         2: 2,  # Cluster 2 corresponds to digit 1
#         3: 0,  # Cluster 3 corresponds to digit 7
#         4: 3,  # Cluster 4 corresponds to digit 4
#         5: 8,  # Cluster 5 corresponds to digit 0
#         6: 9,  # Cluster 6 corresponds to digit 9
#         7: 4,  # Cluster 7 corresponds to digit 3
#         8: 1,  # Cluster 8 corresponds to digit 6
#         9: 0  # Cluster 9 corresponds to digit 8
#     }
    
#     # Apply the mapping
#     updated_labels = modifier.map_cluster_to_digit(cluster_to_digit_map)
    
#     # Optional: Manually correct a few samples
#     # updated_labels = modifier.manually_correct_samples(n_samples=10)
    
#     # Visualize updated labels to confirm changes
#     modifier.visualize_current_labels(samples_per_cluster=5, save_path=vis_path.replace('current', 'updated'))
    
#     # Save the updated dataset
#     modifier.save_updated_dataset(output_path)
    
#     # Verify saved data
#     with np.load(output_path) as data:
#         saved_images = data['images']
#         saved_labels = data['labels']
#         print(f"Saved data: images shape {saved_images.shape}, labels shape {saved_labels.shape}")


def postProcessImages():
    npz_path = '../datasets/labeled_train_data.npz'
    output_path = '../datasets/updated_labeled_train_data.npz'
    
    modifier = ImageLabelModifier(npz_path, random_state=42)
    
    # Iteratively update labels
    updated_labels = modifier.iterative_update_labels(samples_per_cluster=5, vis_path_prefix='labels_visualization')
    
    # Save the updated dataset
    modifier.save_updated_dataset(output_path)
    
    # Verify saved data
    with np.load(output_path) as data:
        saved_images = data['images']
        saved_labels = data['labels']
        print(f"Saved data: images shape {saved_images.shape}, labels shape {saved_labels.shape}")

postProcessImages()